In [11]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Consts 
k1 = 10**(-1.47)
k2 = 10**(-2.59)
k3 = 10**(-3.76)
k4 = 10**(-10.329)
yppm = 415e-06

In [45]:
# Eq Consts Only
ph = 5.5 # Avoid if possible
cb = k1 * yppm * 1
cd = k2 * cb
ce = 10**(-ph)
cf = k3 * cd / ce
cg = k4 * cf / ce
ct = cb + cf + cg
print('xCO2 {:.3g}, xHCO3 {:.3g}, xCO3 {:.3g}'.format(cb/ct, cf/ct, cg/ct))

## functionalized
def phK(pH): 
    global k1, k2, k3, k4, yppm
    cb = k1 * yppm * 1
    cd = k2 * cb
    ce = 10**(-pH)
    cf = k3 * cd / ce
    cg = k4 * cf / ce
    ct = cb + cf + cg
    xs = [cb/ct, cf/ct, cg/ct]
    return xs

phs = np.linspace(0, 14, 100)
xs = phK(phs)
plt.plot(phs, xs[0], label='cCO2') 
plt.plot(phs, xs[1], label='cHCO3') 
plt.plot(phs, xs[2], label='cCO3') 
plt.scatter(ph, cb/ct, c='C0')
plt.annotate('test', (ph, cb/ct), xytext =(ph, cb/ct), xycoords='data', textcoords=0.5)
plt.scatter(ph, cf/ct, c='C1')
plt.scatter(ph, cg/ct, c='C2')

plt.xlim(0, 14)
plt.legend(frameon=False)
plt.show()

xCO2 0.876, xHCO3 0.124, xCO3 1.83e-06


RuntimeError: Unknown coordinate type: 0.5

<Figure size 640x480 with 1 Axes>